In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

In [ ]:
def GetNewton(f,df,xn,itmax=1000,precision=1e-12):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [ ]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)
        
        if root != False:
            
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [ ]:
n=10

Roots1,Weights1=np.polynomial.laguerre.laggauss(n)

x=sym.Symbol("x",real=True)
y=sym.Symbol("y",real=True)

In [ ]:
def GetLaguerre(n,x,y):
    
    y=(x**n)*(sym.exp(-x))
    
    pol=sym.diff(y,x,n)/(np.math.factorial(n))
    
    return pol*(sym.exp(x))

In [ ]:
Laguerre=[]
DevLaguerre=[]

for i in range(n+2):
    Pol=GetLaguerre(i,x,y)
    Laguerre.append(Pol)
    DevLaguerre.append(sym.diff(Pol,x,1))

In [ ]:
def GetAllRoots(grado,xn,Laguerre,DevLaguerre):
    
    pol = sym.lambdify([x],Laguerre[grado],'numpy')
    Dpol = sym.lambdify([x],DevLaguerre[grado],'numpy')
    Roots = GetRoots(pol,Dpol,xn)
    
    return Roots

In [ ]:
xn = np.linspace(0,100,100)

In [ ]:
def GetWeights(Roots,Laguerre,grado):
    
    Pol = sym.lambdify([x],Laguerre[grado+1],'numpy')
    Weights= Roots/(((grado+1)**2)*Pol(Roots)**2)
    
    return Weights

In [ ]:
a=0
b=100
f=lambda x:(x**3)/(np.exp(x)-1)

In [ ]:
def integrate_Laguerre(f,a,b,grado):
    
    Rootsi=GetAllRoots(grado,xn,Laguerre,DevLaguerre)
    
  
    Weightsi=GetWeights(Rootsi,Laguerre,grado)
    
    
    t=Rootsi+a

    integral=np.sum(Weightsi*np.exp(t)*f(t))
    
    return integral

In [ ]:
Integral_n_3=integrate_Laguerre(f,a,b,3)

In [ ]:
print("La aproximacion cuando tenemos n=3 es:",Integral_n_3)

In [ ]:
I=(np.pi**4)/15
error=np.array([])
n_=np.arange(2,n+1)
for i in range(2,n+1):
    integral=integrate_Laguerre(f,a,b,i)
    
    error=np.append(error,integral/I)

In [ ]:
plt.axhline(y=1,color="r",linestyle="--")
plt.grid(visible=True)
plt.scatter(n_,error,color="b",label="Precision de cuadratura de Laguerre")
plt.legend()